https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [76]:
import sqlite3

In [77]:
conn = sqlite3.connect('database/aluar.db')
conn.execute('DROP TABLE IF EXISTS User')
conn.commit()

In [78]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

In [79]:
from sqlalchemy import create_engine
eng = create_engine('sqlite:////Users/julian/Documents/repos/aluar/sql/database/aluar.db')

# Defino modelo

In [80]:
Base = declarative_base()

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name='{self.name}', fullname='{self.fullname}', nickname='{self.nickname}')>"

# Creo la tabla

In [81]:
# nos crea automaticamente el Schema
User.__table__ 

Table('user', MetaData(bind=None), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(), table=<user>), Column('fullname', String(), table=<user>), Column('nickname', String(), table=<user>), schema=None)

In [82]:
Base.metadata.create_all(eng)

In [83]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [84]:
ed_user.name, ed_user.fullname, ed_user.nickname, ed_user.id

('ed', 'Ed Jones', 'edsnickname', None)

# Creación de la sessión

In [85]:
from sqlalchemy.orm import sessionmaker
session = sessionmaker(bind=eng)()

# Agregamos usuario

In [86]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)

# Query

In [87]:
q = session.query(User).filter_by(name='ed')

In [88]:
# all, count, first
q.first()

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [89]:
session.query(User).count()

1

# Agregamos mas usuarios

In [90]:
# Agregamos otros usuarios
session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy'),
                 User(name='mary', fullname='Mary Contrary', nickname='mary'),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [91]:
# Le modificamos el nombre a ed
ed_user.nickname = 'eddie'

In [92]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [93]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [94]:
session.commit()

In [95]:
session.dirty

IdentitySet([])

# Volviendo atras (Rollback)

In [96]:
ed_user.name = 'Edwardo'

In [97]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [98]:
session.dirty

IdentitySet([<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>])

In [99]:
session.new

IdentitySet([<User(name='fakeuser', fullname='Invalid', nickname='12345')>])

In [100]:
session.rollback()

In [101]:
session.dirty

IdentitySet([])

In [102]:
session.new

IdentitySet([])

# Ordenando

In [103]:
for instance in session.query(User).order_by(User.name):
    print(instance.name, instance.fullname)

ed Ed Jones
fred Fred Flintstone
mary Mary Contrary
wendy Wendy Williams
